In [2]:
from agents import Agent, Runner
from dotenv import load_dotenv
from openai.types.responses import ResponseTextDeltaEvent
load_dotenv()

True

In [9]:
agent = Agent(
    name="Joker",
    instructions="You are a helpful assistant",
)

result = Runner.run_streamed(agent, input="Tell me 5 jokes.",)
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Of course! Here are 5 jokes for you:

1. Why don't skeletons fight each other?  
   They don't have the guts.

2. Why did the scarecrow win an award?  
   Because he was outstanding in his field!

3. Why did the bicycle fall over?  
   Because it was two-tired.

4. Parallel lines have so much in common.  
   It’s a shame they’ll never meet.

5. What do you call fake spaghetti?  
   An impasta!

### Example 2

In [17]:
import random
from agents import Agent, ItemHelpers, Runner, function_tool

@function_tool
def how_many_jokes() -> int:
    return random.randint(1, 10)

agent = Agent(
    name="Joker",
    instructions="First call the `how_many_jokes` tool, then tell that many jokes.",
    tools=[how_many_jokes],
)
result = Runner.run_streamed(
    agent,
    input="Hello",
)
print("=== Run starting ===")
async for event in result.stream_events():
    # We'll ignore the raw responses event deltas
    if event.type == "raw_response_event":
        continue
    # When the agent updates, print that
    elif event.type == "agent_updated_stream_event":
        print(f"Agent updated: {event.new_agent.name}")
        continue
    # When items are generated, print them
    elif event.type == "run_item_stream_event":
        if event.item.type == "tool_call_item":
            print("-- Tool was called")
        elif event.item.type == "tool_call_output_item":
            print(f"-- Tool output: {event.item.output}")
        elif event.item.type == "message_output_item":
            print(f"-- Message output:\n {ItemHelpers.text_message_output(event.item)}")
        else:
            pass  # Ignore other event types
print("=== Run complete ===")


=== Run starting ===
Agent updated: Joker
-- Tool was called
-- Tool output: 10
-- Message output:
 Hi there! I have 10 jokes to share with you. Ready? Here we go:

1. Why don't scientists trust atoms? Because they make up everything!
2. Parallel lines have so much in common. It’s a shame they’ll never meet.
3. Why did the scarecrow win an award? Because he was outstanding in his field!
4. What do you call fake spaghetti? An impasta!
5. Why was the math book sad? Because it had too many problems.
6. How does a penguin build its house? Igloos it together.
7. Why did the bicycle fall over? Because it was two-tired!
8. I told my wife she was drawing her eyebrows too high. She looked surprised.
9. What do you call cheese that isn't yours? Nacho cheese!
10. Why couldn't the leopard play hide and seek? Because he was always spotted!

Let me know if you want to hear more, or if you have a favorite!
=== Run complete ===
